<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 798ms/step - loss: 0.6552 - accuracy: 0.6168 - val_loss: 0.6130 - val_accuracy: 0.7073
Epoch 2/10
26/26 [==============================] - 20s 761ms/step - loss: 0.6026 - accuracy: 0.6849 - val_loss: 0.5966 - val_accuracy: 0.6829
Epoch 3/10
26/26 [==============================] - 20s 754ms/step - loss: 0.5664 - accuracy: 0.7141 - val_loss: 0.6571 - val_accuracy: 0.5512
Epoch 4/10
26/26 [==============================] - 20s 766ms/step - loss: 0.5035 - accuracy: 0.7616 - val_loss: 0.4680 - val_accuracy: 0.8049
Epoch 5/10
26/26 [==============================] - 20s 753ms/step - loss: 0.4434 - accuracy: 0.8041 - val_loss: 0.4121 - val_accuracy: 0.8146
Epoch 6/10
26/26 [==============================] - 20s 761ms/step - loss: 0.4130 - accuracy: 0.8175 - val_loss: 0.3617 - val_accuracy: 0.8634
Epoch 7/10
26/26 [==============================] - 20s 758ms/step - loss: 0.3648 - accuracy: 0.8382 - val_loss: 0.3174 - val_accuracy: 0.8976

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.65.h5
26/26 - 20s - loss: 0.6717 - accuracy: 0.5791 - val_loss: 0.6473 - val_accuracy: 0.8000
Epoch 2/5

Epoch 00002: saving model to weights.02-0.59.h5
26/26 - 20s - loss: 0.6254 - accuracy: 0.7141 - val_loss: 0.5889 - val_accuracy: 0.8195
Epoch 3/5

Epoch 00003: saving model to weights.03-0.63.h5
26/26 - 20s - loss: 0.5752 - accuracy: 0.7652 - val_loss: 0.6318 - val_accuracy: 0.5659
Epoch 4/5

Epoch 00004: saving model to weights.04-0.63.h5
26/26 - 20s - loss: 0.5040 - accuracy: 0.8054 - val_loss: 0.6324 - val_accuracy: 0.5902
Epoch 5/5

Epoch 00005: saving model to weights.05-0.35.h5
26/26 - 20s - loss: 0.4332 - accuracy: 0.8187 - val_loss: 0.3531 - val_accuracy: 0.9171


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6675 - accuracy: 0.5985 - val_loss: 0.6612 - val_accuracy: 0.5707


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6730 - accuracy: 0.5900 - val_loss: 0.6298 - val_accuracy: 0.7463
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6092 - accuracy: 0.6898 - val_loss: 0.5687 - val_accuracy: 0.8098


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6833 - accuracy: 0.5827 - val_loss: 0.6551 - val_accuracy: 0.6146
Epoch 2/50
26/26 - 20s - loss: 0.6318 - accuracy: 0.6764 - val_loss: 0.6454 - val_accuracy: 0.5707
Epoch 3/50
26/26 - 20s - loss: 0.6013 - accuracy: 0.7080 - val_loss: 0.5607 - val_accuracy: 0.7707
Epoch 4/50
26/26 - 20s - loss: 0.5801 - accuracy: 0.7251 - val_loss: 0.5155 - val_accuracy: 0.8146
Epoch 5/50
26/26 - 20s - loss: 0.5293 - accuracy: 0.7421 - val_loss: 0.4881 - val_accuracy: 0.8293
Epoch 6/50
26/26 - 21s - loss: 0.4658 - accuracy: 0.7993 - val_loss: 0.6965 - val_accuracy: 0.5854
Epoch 7/50
26/26 - 20s - loss: 0.4141 - accuracy: 0.8285 - val_loss: 0.4063 - val_accuracy: 0.8488
Epoch 8/50
26/26 - 21s - loss: 0.3782 - accuracy: 0.8601 - val_loss: 0.3169 - val_accuracy: 0.8927
Epoch 9/50
26/26 - 20s - loss: 0.3175 - accuracy: 0.8844 - val_loss: 0.4298 - val_accuracy: 0.7463
Epoch 10/50
26/26 - 21s - loss: 0.2699 - accuracy: 0.9027 - val_loss: 0.2106 - val_accuracy: 0.9415
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 22s 831ms/step - loss: 0.6682 - accuracy: 0.5925 - val_loss: 0.6592 - val_accuracy: 0.5951
Epoch 2/5
26/26 [==============================] - 21s 807ms/step - loss: 0.6334 - accuracy: 0.6849 - val_loss: 0.6309 - val_accuracy: 0.6049
Epoch 3/5
26/26 [==============================] - 21s 800ms/step - loss: 0.5753 - accuracy: 0.7348 - val_loss: 0.9050 - val_accuracy: 0.4634
Epoch 4/5
26/26 [==============================] - 21s 823ms/step - loss: 0.5725 - accuracy: 0.7056 - val_loss: 0.5834 - val_accuracy: 0.6390
Epoch 5/5
26/26 [==============================] - 20s 757ms/step - loss: 0.5187 - accuracy: 0.7518 - val_loss: 0.4503 - val_accuracy: 0.8829


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.592457,0.668551,0.595122,0.659243
1,1,0.684915,0.633992,0.604878,0.630897
2,2,0.734793,0.576918,0.463415,0.905005
3,3,0.705596,0.573274,0.639024,0.583387
4,4,0.751825,0.520045,0.882927,0.450344


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 31s 1s/step - loss: 0.6797 - accuracy: 0.5438 - val_loss: 0.6702 - val_accuracy: 0.6390

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 807ms/step - loss: 0.6622 - accuracy: 0.6058 - val_loss: 0.6637 - val_accuracy: 0.6195

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 765ms/step - loss: 0.6514 - accuracy: 0.6302 - val_loss: 0.6633 - val_accuracy: 0.5512

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 777ms/step - loss: 0.6460 - accuracy: 0.6338 - val_loss: 0.6599 - val_accuracy: 0.5805

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 766ms/step - loss: 0.6438

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 857ms/step - loss: 0.6878 - accuracy: 0.5462 - val_loss: 0.6879 - val_accuracy: 0.4780
Epoch 2/50
26/26 [==============================] - 21s 800ms/step - loss: 0.6185 - accuracy: 0.6825 - val_loss: 0.6027 - val_accuracy: 0.6927
Epoch 3/50
26/26 [==============================] - 21s 808ms/step - loss: 0.5879 - accuracy: 0.6776 - val_loss: 0.5627 - val_accuracy: 0.7366
Epoch 4/50
26/26 [==============================] - 20s 784ms/step - loss: 0.5470 - accuracy: 0.7214 - val_loss: 0.4878 - val_accuracy: 0.8146
Epoch 5/50
26/26 [==============================] - 21s 812ms/step - loss: 0.4897 - accuracy: 0.7786 - val_loss: 0.4282 - val_accuracy: 0.8976
Epoch 6/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4272 - accuracy: 0.8438
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 786ms/step - loss: 0.4234 - accuracy: 0.8467 - val_loss: 0.4455 - 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4637), started 0:17:03 ago. (Use '!kill 4637' to kill it.)